In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Ensemble
- 여러개의 분류 모델을 조합해서 더 나은 성능을 내는 방법     
- Decision Tree Model을 증가시켜 나온 Random Forest가 대표적임.    

### Random Forest
- used bootstrap sample.   
    - bootstrap sampling = 중복을 허용하는 sampling
- sampling 후에 sample을 다시 복구하고 다시 sampling 하는 방법
- 이와 같이 진행하는 이유는 DecisionTree 에서 과대적합을 방지 할 수 있기 때문.
- 각 DecisionTree에서 나오는 확률의 합을 Tree갯수로 나누어 결정짓는 Model
- 특성의 갯수를 제곱근(루트) 으로 sampling 한다

In [3]:
wine = pd.read_csv('../Data/wine.csv')

# Feature n Target
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

# # Train n Test
# train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
# sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

# Train n Test
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# Random Forest
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1, random_state =42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [5]:
# important Feature check
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [6]:
# bootstrap 결정 시 남는 sample(oob:out of back)도 특성으로 구분할 수 있다.

rf = RandomForestClassifier(oob_score=True, n_jobs = -1, random_state=42)
rf.fit(train_input,train_target)
rf.oob_score_

0.8934000384837406

> oob를 사용해도 90%의 예측력이 나온다      
> 따로 검증셋을 구성하지 않아도 oob 가 valid 의 역활을 할 수 있다. 

> RandomForest는 기본값 만으로도 높은 성능을 발휘하므로 자주 사용되는 Model(algorithm)중 하나 이다

---
### Extra Tree
- 기본적으로 100개의 트리를 사용
- 노드 분할시 특성(Feature, column)의 제곱근을 갯수로 사용
- 특성(Feature, column)의 선택을 랜덤하게 선택함 (
- 특성의 선택을 랜덤하게 하므로 속도는 RandomForest보다 빠름 
    (column을 제곱근해서 잘라서 쓰니까 rf보다 당연히 빠름)

In [7]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs= -1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [8]:
et.fit(train_input,train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

---
### Gradient Boosting
- 가장 유명한 알고리즘 중 하나
- 경사하강법 처럼 손실함수를 사용
- 손실함수를 보고 트리를 추가하여 최적의 값 도출하는 방법
- 경사를 이동하면서 경사의 이동거리를 제어하는 Learning-rate(default:0.1)를 사용한다ㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏ(증가율)
- max depth를 3으로 제어하여 깊이가 낮으므로 과대적합 방지
- 단점 : 손실함수를 보고 트리를 추가하면서 진행하는 모델이므로 병렬처리를 할 수 없다. (n_jobs=False)

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.8881086892152563 0.8720430147331015


tree 추가를 500개까지 하고, learning rate를 0.2로 변경

In [11]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [12]:
gb.fit(train_input,train_target)
gb.feature_importances_

array([0.15872278, 0.68010884, 0.16116839])

---
### Histogram GradientBoosting
- 훈련데이터를 256개 구간으로 나누어서 훈련시키는 방법
- 특성의 범위가 제한되어 있어 빠른 속도를 제공한다
- 제한된 구간이므로 과대적합을 방지한다
- 아직은 실험단계인 모델

In [13]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


## XGBoost
- kaggle에서 많이 사용

In [14]:
# !pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.1 MB/s eta 0:00:00a 0:00:01


In [18]:
from xgboost import XGBClassifier
xgb = XGBClassifier(
    tree_method = 'hist', 
    random_state=42,
    # user_label_encoder = False,
    # eval_metrics = 'logloss' # Histogram Gradient Boosting
)

In [19]:
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


---
# 번외 기능
#### Permutation Importance(치환 중요도)
- 각 Feature별 Sample(data)을 섞어서 계산을 한 후 에 원래 Sample 들과의 차이를 계산해서 차이가 많이 나는 Feature가 중요하다고 판단.
- 즉, 어느 Feature가 중요한지 파악하는 방법
- 어떤 모델에도 사용가능하며 특성을 파악하는 주요 기준으로 사용된다. (> * 권장사항 * <)

In [24]:
from sklearn.inspection import permutation_importance

xgb.fit(train_input, train_target)
result = permutation_importance(xgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
result.importances_mean #= array([0.11306523, 0.25508947, 0.10696556])


array([0.11306523, 0.25508947, 0.10696556])

In [25]:

gb.fit(train_input, train_target)
result = permutation_importance(gb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
result.importances_mean #= array([0.09661343, 0.24265923, 0.09120647])


array([0.09661343, 0.24265923, 0.09120647])

기존의 중요도에서 permutation_importance를 사용했을시 가장 많이 떨어지는 값이 가장 중요한 Feature이다

In [26]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [27]:
rf.fit(train_input, train_target)
result = permutation_importance(rf, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
result.importances_mean

array([0.16147778, 0.29559361, 0.15730229])

---
Ensanble 정리
- ensanble 학습은 정형데이터에서 가장 뛰어난 성능을 내는 ML Algorithm 중 하나 이다.

#### Random Forest
: bootstrap sample 사용, 대표 ensanbel 학습 알고리즘

#### Extra Tree
: DecisionTree의 노드를 랜덤하게 분할함

#### Gradient Boosting
: 트리의 손실을 보완하는 식으로 얕은 DecisionTree를 연속하여 추가함

#### Hostogram Gradient Boosting
: 훈련 데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄

#### 기타
- SGBoost